In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()
from src.summarize_text import *


Selected device: cuda


In [3]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Selected device: cuda


In [4]:
text

"1: I'll be home later at 8 o'clock to pack my clothes\n\n2: At 9 o'clock I go to bed to write my essay and at 10 o'clock I go to bed\n"

In [5]:
symbol = 'SSI'

In [6]:


class NewsScraper:
    '''
    Scape News from https://vnexpress.net/ 
    How to run selenium on linux
    https://cloudbytes.dev/snippets/run-selenium-and-chrome-on-wsl2#:~:text=With%20Selenium%20libraries%2C%20Python%20can,using%20Python%20and%20Selenium%20webdriver.
    
    '''
    def __init__(self):
        pass

    def search_stock_news(self, symbol:str = "SSI",
                          date_format:Literal['day', 'week', 'month', 'year']='day')-> list : 
        symbol = symbol.upper()
        url = f"https://timkiem.vnexpress.net/?search_f=&q={symbol}&date_format={date_format}&"

        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Assuming the news articles are wrapped in <article> elements
                articles = soup.find_all('article', class_='item-news-common')
                # Iterate through each article and extract the URL
                for article in articles:
                    url = article.get('data-url')
                    if url:
                        news_urls.append(url)

                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls

    def search_top_news_cafef(self)-> list:

        url = 'https://cafef.vn/'
        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find top news elements
                top_news_elements = soup.find_all('div', class_='top_noibat')
                top_news = top_news_elements[0] if top_news_elements else None

                top_news_links = top_news.find_all('a')
                # Extract and print the links
                for link in top_news_links:
                    news_link = link.get('href')
                    news_link = f'{url}{news_link}'
                    news_urls.append(news_link)

                    # Convert set to list
                    news_urls = list(set(news_urls))
                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls
    
    def search_top_news_vnexpress(self)-> list:

        url = 'https://vnexpress.net/kinh-doanh'
        # Send a GET request to the webpage
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        attemp = 0
        max_attemps = 3
        news_urls = []

        while attemp <= max_attemps:
            try:
                # Parse the HTML content
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find news elements
                news_elements = soup.find_all('h3', class_='title_news')
                news_elements.append(soup.find('section', class_='section_topstory_folder'))

                # Extract and return the URLs
                news_urls = [element.find('a').get('href') for element in news_elements if element.find('a')]
                break
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                print("Resetting the Scraper in 10 seconds...")
                time.sleep(10)  
                attemp += 1
                if attemp > max_attemps:
                    print("Max attempts reached. Exiting.")
                    break

        return news_urls
    
    def search_top_news(self)-> list:
        news_urls = self.search_top_news_cafef()
        news_urls.extend(self.search_top_news_vnexpress())
        return news_urls
    
    def take_text_from_link(self, news_url:str) -> str :
        news_url = [news_url]
        loader = NewsURLLoader(urls=news_url, 
                            post_processors=[clean_extra_whitespace],)
        news_text = loader.load()
        news_text = news_text[0].page_content
        return  news_text

In [7]:
news_scraper = NewsScraper()
# news_list = news_scraper.search_stock_news(symbol=symbol, date_format='year')
news_list = news_scraper.search_top_news()
news = news_scraper.take_text_from_link(news_url=news_list[0])

In [8]:
news

'Nhà đầu tư chứng khoán tại Việt Nam đang có một khoảng thời gian tươi đẹp khi thị trường liên tục đi lên thời gian gần đây. Sau hơn 2 tháng, VN-Index tăng hơn 12% qua đó leo lên mức cao nhất kể từ giữa tháng 10 năm ngoái. Mặc dù không quá bùng nổ nhưng diễn biến khởi sắc này cũng gợi nhớ phần nào đến giai đoạn tuyệt vời cách đây 2 năm.\n\nĐúng vào ngày này (6/1) năm 2022, thị trường chứng khoán Việt Nam ghi dấu một cột mốc quan trọng khi VN-Index lập đỉnh lịch sử 1.528,57 điểm. Đến nay, đây vẫn là mức cao nhất mà chỉ số này từng chạm đến. Vốn hóa HoSE thời điểm đó đạt xấp xỉ 6 triệu tỷ, giá trị toàn sàn chứng khoán lên đến gần 8 triệu tỷ đồng.\n\nThành tích “vô tiền khoáng hậu” trên là kết quả của một con sóng thần với động lực chính là sự đổ bộ ồ ạt của các nhà đầu tư mới trong môi trường tiền rẻ. Sau gần 2 năm kể từ đáy Covid (cuối tháng 3/2020), VN-Index đã tăng hơn 120%. Có không ít “ổ gà” trên đường lên đỉnh lịch sử nhưng nhìn chung xu hướng tăng vẫn xuyên suốt giai đoạn đó.\n\nS

In [9]:
new_summarizer = NewsSummarizer()
sum_text = new_summarizer.summary_news(news= news)

Your max_length is set to 230, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)


In [10]:
print(sum_text)

Sau hơn 2 tháng, VN-Index tăng hơn 12%, qua đó leo lên mức cao nhất kể từ giữa tháng 10 năm ngoái. 
Mức cao nhất mà chỉ số đạt được trong hơn một năm là dưới 1.250 điểm. 
Sự đi lên này phần nào gợi nhớ lại khoảng thời gian tuyệt vời cách đây 2 năm.

Các quỹ ngoại cho rằng thị trường có tiềm năng quay trở lại mức định giá P/S trên 2, tương đương VN-Index ở đỉnh 1.500 điểm trong 12 tháng tới. 
Con số doanh thu của các công ty niêm yết tại Việt Nam sẽ đạt kỷ lục vào năm 2024, tốc độ tăng trưởng hơn 20%

Hàng tỷ USD vốn ngoại sẽ đổ vào chứng khoán Việt Nam nếu thị trường được nâng hạng từ cận biên lên mới nổi . 
nếu được nâng hạng sẽ góp phần hỗ trợ thị trường chứng khoán trong thời gian tới.



In [11]:
watch_list=['GEX', 'VIX', 'HHV', 'BSR', 'SHS', 'PDR', 'CTR', 'HAH', 'VNINDEX', 'VND']

In [22]:
news_db = StockNewsDatabase()
# news_db.update_stock_news(watch_list)

In [13]:
# news_db.update_news(watch_list=watch_list)

In [14]:
news_db.extract_text_for_stock(stock_symbol='HAH')

('Cổ phiếu GMD của Gemadept là một trong những cổ phiếu tăng mạnh nhất sàn HoSE khi đóng cửa tăng toàn bộ lên 73.000 đồng. \nCác cổ phiếu cảng biển, logistics khác cũng giao dịch sôi động như HAH tăng gần 4%. \nChứng khoán cũng mở cửa phiên đầu tiên trong trạng thái tích cực.\n\nỞ nhóm ngân hàng, sắc xanh cũng chiếm ưu thế. \nVPB, BID, CTG tăng gần 2%, VIB, HDB, TCB, STB tăng trên 1%. \nNhóm bán lẻ và hàng tiêu dùng cũng nhận được sự quan tâm.\n',
 'https://vnexpress.net/co-phieu-van-tai-bien-noi-song-4693192.html')

In [15]:
news_db.get_all_stocks()

['HAH', 'VIX']

In [23]:
news_db.get_top_news()

'Sau hơn 2 tháng, VN-Index tăng hơn 12%, qua đó leo lên mức cao nhất kể từ giữa tháng 10 năm ngoái. \nMức cao nhất mà chỉ số đạt được trong hơn một năm là dưới 1.250 điểm. \nSự đi lên này phần nào gợi nhớ lại khoảng thời gian tuyệt vời cách đây 2 năm.\n\nCác quỹ ngoại cho rằng thị trường có tiềm năng quay trở lại mức định giá P/S trên 2, tương đương VN-Index ở đỉnh 1.500 điểm trong 12 tháng tới. \nCon số doanh thu của các công ty niêm yết tại Việt Nam sẽ đạt kỷ lục vào năm 2024, tốc độ tăng trưởng hơn 20%\n\nHàng tỷ USD vốn ngoại sẽ đổ vào chứng khoán Việt Nam nếu thị trường được nâng hạng từ cận biên lên mới nổi . \nnếu được nâng hạng sẽ góp phần hỗ trợ thị trường chứng khoán trong thời gian tới.\n\nhttps://cafef.vn//ngay-nay-2-nam-truoc-vn-index-lap-dinh-lich-su-152857-diem-188240105233819823.chn\n----------\nSố liệu từ Hiệp hội môi giới bất động sản Việt Nam cho thấy, năm 2022, nguồn cung nhà ở ra thị trường sẽ đạt khoảng 43.500 căn hộ, tương đương 90% tổng số sản phẩm chào bán năm 